<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/CNN-inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, Dense, AveragePooling1D
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [4]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv',chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [5]:
model = Sequential()
model.add(Conv1D(activation="sigmoid", input_shape=(len(test.columns)-1,1), kernel_size=2, filters=100))
model.add(AveragePooling1D(pool_size=2))
model.add(Dense(1))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 69, 100)           300       
                                                                 
 average_pooling1d (AverageP  (None, 34, 100)          0         
 ooling1D)                                                       
                                                                 
 dense (Dense)               (None, 34, 1)             101       
                                                                 
Total params: 401
Trainable params: 401
Non-trainable params: 0
_________________________________________________________________


In [6]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x,train_y,batch_size=32,epochs=6)

Epoch 1/6
15625/15625 [==============================] - 53s 3ms/step - loss: 0.6972 - accuracy: 0.5007
Epoch 2/6
15625/15625 [==============================] - 41s 3ms/step - loss: 0.6971 - accuracy: 0.5012
Epoch 3/6
15625/15625 [==============================] - 39s 2ms/step - loss: 0.6970 - accuracy: 0.5007
Epoch 4/6
15625/15625 [==============================] - 39s 2ms/step - loss: 0.6964 - accuracy: 0.5018
Epoch 5/6
15625/15625 [==============================] - 41s 3ms/step - loss: 0.6963 - accuracy: 0.5019
Epoch 6/6
15625/15625 [==============================] - 38s 2ms/step - loss: 0.6964 - accuracy: 0.5014
Epoch 1/6
15625/15625 [==============================] - 39s 2ms/step - loss: 0.6959 - accuracy: 0.5022
Epoch 2/6
15625/15625 [==============================] - 38s 2ms/step - loss: 0.6959 - accuracy: 0.5007
Epoch 3/6
15625/15625 [==============================] - 38s 2ms/step - loss: 0.6958 - accuracy: 0.5020
Epoch 4/6
15625/15625 [==============================] - 39s 2ms

In [7]:
test_y = test.pop('Label')
test_x = test

In [8]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [9]:
score,accuracy = model.evaluate(test_x,test_y)
print('Score',score,"Accuracy",accuracy)

79666/79666 [==============================] - 209s 3ms/step - loss: 0.6932 - accuracy: 0.5074
Score 0.6931599378585815 Accuracy 0.5074174404144287
